# 07 - Hyperparameter Sweeps

Optimize models with Weights & Biases sweeps.

## Learning Objectives
- Configure W&B sweeps
- Run Bayesian optimization
- Analyze sweep results
- Select best configurations

In [ ]:
# Setup
import wandb
import yaml
from pathlib import Path

project_root = Path.cwd().parent
print(f'Project root: {project_root}')

## 1. Sweep Configuration

View the sweep config:

In [ ]:
# Load sweep config
config_path = project_root / 'configs' / 'sweep_config.yaml'

with open(config_path, 'r') as f:
    sweep_config = yaml.safe_load(f)

print('Sweep method:', sweep_config['method'])
print('Metric:', sweep_config['metric'])
print('Parameters:', list(sweep_config['parameters'].keys()))

## 2. Start Sweep

Command-line sweep:

```bash
# Create sweep
wandb sweep configs/sweep_config.yaml

# Run agents
wandb agent YOUR_ENTITY/PROJECT/SWEEP_ID
```

## 3. Analyze Results

After sweep completes, analyze results:

In [ ]:
# Example: Fetch sweep results via W&B API
# sweep_id = 'YOUR_SWEEP_ID'
# api = wandb.Api()
# sweep = api.sweep(f'{entity}/{project}/{sweep_id}')
# runs = sweep.runs

print('Use scripts/analyze_sweep.py for detailed analysis')

## 4. Post-Sweep Workflow

After sweep:

```bash
./scripts/post_sweep_workflow.sh \\
    --sweep-id SWEEP_ID \\
    --entity ENTITY \\
    --deploy
```

## Summary

You learned:
- Configuring sweeps
- Running Bayesian optimization
- Analyzing results
- Post-sweep workflow

## Next Steps

Continue to **08_model_evaluation.ipynb** for comprehensive model evaluation.